In [ ]:
!pip install optuna
#!pip install joblib

In [79]:
import optuna
import joblib
import pandas as pd
import numpy as np
import random
import math

In [80]:
def BALANCE(weights):
  #Making sure the total sum of the weights eual to 1
  weights = [w/sum(weights) for w in weights] 
  # Making sure all weights represent proportions that add up to 1
  return weights

In [81]:
def ratio(a,b,c):                       
  #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
  #calculating sharpe ratio
  return (a-c)/b

In [82]:
def number_of_years(y):#calculates the number of years of the dataset
  p=y.index[0]         #date of first row in the dataset (datetime format)
  q=y.index[len(y)-1]  #date of last row in the dataset  (datetime format)
  return ((q-p).days+1)/365

In [83]:
df=pd.read_csv("n50.csv",parse_dates=['Date'],index_col='Date')  #Importing Dataset
df = df.loc["2016-01-01" : ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)
col=list(df.columns)

In [84]:
trading_days=len(df)/number_of_years(tdf) 

In [85]:
returnsh=df.pct_change()                  
#Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)

In [86]:
returnso = returnsh.copy()  # this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
  for i in range(0, len(df)):
    if returnso[cols][i] > 0:
      returnso[cols][i] = 0


In [87]:
covmatsh=returnsh.cov()*trading_days     
#Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso = returnso.cov() * trading_days

In [88]:
risk_free_rate = 0.0358 #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [89]:
df
stocks=df.shape[1]
stocks

22

# Sharpe


In [90]:
def antcolony_tuning_sharpe(ITERATIONS,Q,EVA_RATE,ANTS):
    sharpe_pbest=-1
    #Initializing sharpe_pbest(the best fitness value   SHARPE)
    #Initializing the current fitness value
    fitness=0
    #for each iteration
    for iteration in range(ITERATIONS):
        
        #PREPARAING THE PHEROMONE MATRIX WHERE THE COLS=STOCKS AND  ROWS=ANTS
        pheromon=[[0]*stocks for i in range(ANTS+1)]#why (ants+1)?The last ant can update the pheromone values in the last row
        
        # Initializing the pheromone status 
        for i in range(len(pheromon[0])):
            pheromon[0][i]=random.randint(1,15)   #When input stocks varies, this needs to vary accordingly.(Divide number of stocks / 2)
        
        #copying the values and storing it in temp_pher
        temp_pher=pheromon[0]
        
        #Making sure that the total amount of pheromone equals 1 
        weights=np.array(BALANCE(temp_pher))
        
        #calculating annulaised portfolio return
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days 
        
        #calculating portfolio varience wrt calculating sharpe ratio
        varsh=np.dot(weights.T,np.dot(covmatsh,weights))   
        
        #portfolio risk
        volatility_temp = np.sqrt(varsh)      
        
        #Calculating fitness value(ie sharpe ratio)
        fitness = ratio(returns_temp,volatility_temp,risk_free_rate)
        
        #Initializing the intial fitness value as the best fitness value(sharpe_pbest)
        if sharpe_pbest==-1:
            sharpe_pbest=fitness
        
        #list
        path=[]
      
        #for each ant
        for ant in range(ANTS-1):
            
            #find the total pheromone 
            total=sum(pheromon[ant])
            
            #Initializing probability
            probability=pheromon[ant][:] 
            
            #finding probability of each stocks pheromone 
            for p in range(len(probability)):
                probability[p]=(probability[p]/total)
                
            #Trying to select stocks in decreasing order based on their pheromone level and storing the stock order in a list(path)
            for stock in range(stocks):
                select=probability.index(max(probability))
                probability[select]=-math.inf
                path.append(select)
            
            #Updating the pheromone level of each stock for the next ant 
            #Formula: old pheromone level * (1-eva_rate) + Q * (fitness/sharpe_pbest) where Q is fixed amount of pheromone
            for s in path:
                pheromon[ant+1][s]=pheromon[ant][s]*(1-EVA_RATE)+Q*(fitness/sharpe_pbest)
            
            
            #making sure that the updated pheromon adds upto 1
            temp_pher=pheromon[ant+1]
            weights=np.array(BALANCE(temp_pher))
            returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
            varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
            volatility_temp = np.sqrt(varsh)                              #portfolio risk
            fitness = ratio(returns_temp,volatility_temp,risk_free_rate)  #calculating sharpe ratio
            
            #comparing the old fitness value with the updated fitness value
            # explore on scape condition paper ref
            if(fitness>sharpe_pbest):
                
                #if the updated fitness value is better than the previous, change sharpe_pbest to present fitness value
                sharpe_pbest=fitness
                
                #remembering the weights of the best portfolio
                global_warr_sharpe=weights.tolist()
        #sharpe_portfolio_return.append(returns_temp)
        #sharpe_portfolio_risk.append(volatility_temp)
        #sharpe_portfolio_shratio.append(fitness)
        #sharpe_portfolio_stockWeights.append(weights)
    return sharpe_pbest


In [ ]:

def objective(trial):
    ITERATIONS=trial.suggest_int('ITERATIONS',2,200)#range is temp,earlier (1,1000)
    Q=trial.suggest_float('Q',0.0,1.0)
    EVA_RATE=trial.suggest_float('EVA_RATE',0.00,1.00)
    ANTS=trial.suggest_int('ANTS',2,200)#range is temp,earlier (1,1000)
    return antcolony_tuning_sharpe(int(ITERATIONS),Q,EVA_RATE,int(ANTS))

In [ ]:
sharpe_study=optuna.create_study(direction='maximize')
sharpe_study.optimize(objective,n_trials=10)

[I 2021-11-10 03:48:19,454] A new study created in memory with name: no-name-5bcba5d5-ba40-481d-aa43-1fd2e8f8bcab
[I 2021-11-10 03:48:22,684] Trial 0 finished with value: 1.3600459105299783 and parameters: {'ITERATIONS': 119, 'Q': 0.5618725567007057, 'EVA_RATE': 0.625465769480512, 'ANTS': 27}. Best is trial 0 with value: 1.3600459105299783.
[I 2021-11-10 03:48:48,551] Trial 1 finished with value: 1.3471462926724607 and parameters: {'ITERATIONS': 56, 'Q': 0.8808157465295087, 'EVA_RATE': 0.964689113629478, 'ANTS': 198}. Best is trial 0 with value: 1.3600459105299783.
[I 2021-11-10 03:49:29,988] Trial 2 finished with value: 1.3547212294170583 and parameters: {'ITERATIONS': 109, 'Q': 0.036679775445278895, 'EVA_RATE': 0.5981053038007321, 'ANTS': 176}. Best is trial 0 with value: 1.3600459105299783.
[I 2021-11-10 03:49:34,589] Trial 3 finished with value: 1.3684154631671308 and parameters: {'ITERATIONS': 37, 'Q': 0.698351105655195, 'EVA_RATE': 0.5393574976548745, 'ANTS': 87}. Best is trial 3

In [ ]:
sh_hptuning=sharpe_study.trials_dataframe()
sh_hptuning.to_csv("sharpe_trial0.csv")
best=sharpe_study.best_params
best

{'ANTS': 131,
 'EVA_RATE': 0.10481788652694468,
 'ITERATIONS': 197,
 'Q': 0.04176612938675081}

In [91]:
ITERATIONS=419#int(best['ITERATIONS'])
Q=0.0400026#best['Q']
EVA_RATE=0.128567#best['EVA_RATE']
ANTS=90#int(best['ANTS'])

In [92]:
global_warr_sortino=[]
global_war_sharpe=[]
sharpe_portfolio_return=[]
sharpe_portfolio_risk=[]
sharpe_portfolio_shratio=[]
sharpe_portfolio_stockWeights=[]

In [112]:
def antcolony_sharpe(ITERATIONS,Q,EVA_RATE,ANTS):
    sharpe_pbest=-1
    #Initializing sharpe_pbest(the best fitness value   SHARPE)
    #Initializing the current fitness value
    fitness=0
    #for each iteration
    for iteration in range(ITERATIONS):
        
        #PREPARAING THE PHEROMONE MATRIX WHERE THE COLS=STOCKS AND  ROWS=ANTS
        pheromon=[[0]*stocks for i in range(ANTS+1)]#why (ants+1)?The last ant can update the pheromone values in the last row
        
        # Initializing the pheromone status 
        for i in range(len(pheromon[0])):
            pheromon[0][i]=random.randint(1,15)   #When input stocks varies, this needs to vary accordingly.(Divide number of stocks / 2)
        
        #copying the values and storing it in temp_pher
        temp_pher=pheromon[0]
        
        #Making sure that the total amount of pheromone equals 1 
        weights=np.array(BALANCE(temp_pher))
        
        #calculating annulaised portfolio return
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days 
        
        #calculating portfolio varience wrt calculating sharpe ratio
        varsh=np.dot(weights.T,np.dot(covmatsh,weights))   
        
        #portfolio risk
        volatility_temp = np.sqrt(varsh)      
        
        #Calculating fitness value(ie sharpe ratio)
        fitness = ratio(returns_temp,volatility_temp,risk_free_rate)
        
        #Initializing the intial fitness value as the best fitness value(sharpe_pbest)
        if sharpe_pbest==-1:
            sharpe_pbest=fitness
        
        #list
        path=[]
      
        #for each ant
        for ant in range(ANTS-1):
            
            #find the total pheromone 
            total=sum(pheromon[ant])
            
            #Initializing probability
            probability=pheromon[ant][:] 
            
            #finding probability of each stocks pheromone 
            for p in range(len(probability)):
                probability[p]=(probability[p]/total)
                
            #Trying to select stocks in decreasing order based on their pheromone level and storing the stock order in a list(path)
            for stock in range(stocks):
                select=probability.index(max(probability))
                probability[select]=-math.inf
                path.append(select)
            
            #Updating the pheromone level of each stock for the next ant 
            #Formula: old pheromone level * (1-eva_rate) + Q * (fitness/sharpe_pbest) where Q is fixed amount of pheromone
            for s in path:
                pheromon[ant+1][s]=pheromon[ant][s]*(1-EVA_RATE)+Q*(fitness/sharpe_pbest)
            
            
            #making sure that the updated pheromon adds upto 1
            temp_pher=pheromon[ant+1]
            weights=np.array(BALANCE(temp_pher))
            returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
            varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
            volatility_temp = np.sqrt(varsh)                              #portfolio risk
            fitness = ratio(returns_temp,volatility_temp,risk_free_rate)  #calculating sharpe ratio
            
            #comparing the old fitness value with the updated fitness value
            # explore on scape condition paper ref
            if(fitness>sharpe_pbest):
                
                #if the updated fitness value is better than the previous, change sharpe_pbest to present fitness value
                sharpe_pbest=fitness
                
                #remembering the weights of the best portfolio
        
                global_warr_sharpe=weights.tolist()
            
            sharpe_portfolio_return.append(returns_temp)
            sharpe_portfolio_risk.append(volatility_temp)
            sharpe_portfolio_shratio.append(fitness)
            sharpe_portfolio_stockWeights.append(weights)
        
    return sharpe_pbest


In [113]:
tuned=antcolony_sharpe(ITERATIONS,Q,EVA_RATE,ANTS)

In [116]:
print(len(waste),len(waste1))

419 37291


In [103]:
tuned

1.3984836967675582

In [96]:
sharpe_portfolio = {'Returns' : sharpe_portfolio_return, 'Standard Deviation' : sharpe_portfolio_risk,  'Sharpe Ratio' : sharpe_portfolio_shratio}  

for counter,symbol in enumerate(df.columns):
  sharpe_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sharpe_portfolio_stockWeights]
sharpe_pc = pd.DataFrame(sharpe_portfolio)
sharpe_optimal=sharpe_pc.iloc[sharpe_pc['Sharpe Ratio'].idxmax()]
sharpe_optimal=pd.DataFrame(sharpe_optimal)
sharpe_optimal.to_csv("sharpe_optimal.csv")

sharpe_optimal

,23674
Returns,0.285329
Standard Deviation,0.178428
Sharpe Ratio,1.398484
ASIANPAINT Weight,0.017405
BAJFINANCE Weight,0.034546
BAJAJFINSV Weight,0.028833
BRITANNIA Weight,0.045974
DIVISLAB Weight,0.085971
HCLTECH Weight,0.051688
HDFCBANK Weight,0.040260


In [97]:
sharpe_pc.shape

(37291, 25)

# not sort

In [53]:
sharpe_unsort_top=sharpe_pc.iloc[:,0:3].head(10)
sharpe_unsort_top.to_csv("sharpe_unsort_top.csv")
sharpe_unsort_top

,Returns,Standard Deviation,Sharpe Ratio
0,0.291061,0.203023,1.257299
1,0.290941,0.202854,1.257757
2,0.290807,0.202664,1.258273
3,0.290655,0.202450,1.258852
4,0.290484,0.202211,1.259500
5,0.290293,0.201942,1.260225
6,0.290079,0.201644,1.261030
7,0.289840,0.201312,1.261921
8,0.289574,0.200945,1.262904
9,0.289280,0.200541,1.263982


In [52]:
sharpe_unsort_top_all=sharpe_pc.head(10)
sharpe_unsort_top_all.to_csv("sharpe_unsort_top_all.csv")
sharpe_unsort_top_all

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,HCLTECH Weight,HDFCBANK Weight,HINDALCO Weight,HINDUNILVR Weight,HDFC Weight,ICICIBANK Weight,INFY Weight,JSWSTEEL Weight,KOTAKBANK Weight,NESTLEIND Weight,RELIANCE Weight,SHREECEM Weight,TCS Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,WIPRO Weight
0,0.291061,0.203023,1.257299,0.086698,0.037318,0.055836,0.006456,0.055836,0.006456,0.012628,0.086698,0.062008,0.018801,0.086698,0.006456,0.055836,0.068180,0.037318,0.080525,0.068180,0.018801,0.080525,0.049663,0.006456,0.012628
1,0.290941,0.202854,1.257757,0.086405,0.037376,0.055762,0.006733,0.055762,0.006733,0.012861,0.086405,0.061890,0.018990,0.086405,0.006733,0.055762,0.068019,0.037376,0.080276,0.068019,0.018990,0.080276,0.049633,0.006733,0.012861
2,0.290807,0.202664,1.258273,0.086074,0.037441,0.055678,0.007046,0.055678,0.007046,0.013125,0.086074,0.061758,0.019204,0.086074,0.007046,0.055678,0.067837,0.037441,0.079995,0.067837,0.019204,0.079995,0.049599,0.007046,0.013125
3,0.290655,0.202450,1.258852,0.085700,0.037515,0.055584,0.007399,0.055584,0.007399,0.013422,0.085700,0.061608,0.019445,0.085700,0.007399,0.055584,0.067631,0.037515,0.079677,0.067631,0.019445,0.079677,0.049561,0.007399,0.013422
4,0.290484,0.202211,1.259500,0.085280,0.037598,0.055479,0.007796,0.055479,0.007796,0.013756,0.085280,0.061439,0.019717,0.085280,0.007796,0.055479,0.067399,0.037598,0.079320,0.067399,0.019717,0.079320,0.049518,0.007796,0.013756
5,0.290293,0.201942,1.260225,0.084809,0.037691,0.055360,0.008242,0.055360,0.008242,0.014132,0.084809,0.061250,0.020021,0.084809,0.008242,0.055360,0.067140,0.037691,0.078919,0.067140,0.020021,0.078919,0.049470,0.008242,0.014132
6,0.290079,0.201644,1.261030,0.084282,0.037795,0.055227,0.008740,0.055227,0.008740,0.014551,0.084282,0.061038,0.020362,0.084282,0.008740,0.055227,0.066849,0.037795,0.078471,0.066849,0.020362,0.078471,0.049416,0.008740,0.014551
7,0.289840,0.201312,1.261921,0.083694,0.037911,0.055079,0.009296,0.055079,0.009296,0.015019,0.083694,0.060802,0.020742,0.083694,0.009296,0.055079,0.066525,0.037911,0.077971,0.066525,0.020742,0.077971,0.049356,0.009296,0.015019
8,0.289574,0.200945,1.262904,0.083040,0.038040,0.054915,0.009914,0.054915,0.009914,0.015539,0.083040,0.060540,0.021164,0.083040,0.009914,0.054915,0.066165,0.038040,0.077415,0.066165,0.021164,0.077415,0.049290,0.009914,0.015539
9,0.289280,0.200541,1.263982,0.082318,0.038182,0.054733,0.010597,0.054733,0.010597,0.016114,0.082318,0.060250,0.021631,0.082318,0.010597,0.054733,0.065767,0.038182,0.076801,0.065767,0.021631,0.076801,0.049216,0.010597,0.016114


In [54]:
sharpe_unsort_bottom=sharpe_pc.iloc[:,0:3].tail(10)
sharpe_unsort_bottom.to_csv("sharpe_unsort_bottom.csv")
sharpe_unsort_bottom

,Returns,Standard Deviation,Sharpe Ratio
37281,0.274301,0.183314,1.301052
37282,0.274300,0.183314,1.301051
37283,0.274300,0.183314,1.301049
37284,0.274300,0.183314,1.301048
37285,0.274300,0.183314,1.301047
37286,0.274299,0.183314,1.301046
37287,0.274299,0.183314,1.301045
37288,0.274299,0.183314,1.301044
37289,0.274299,0.183314,1.301043
37290,0.274299,0.183314,1.301043


In [55]:
sharpe_unsort_bottom_all=sharpe_pc.tail(10)
sharpe_unsort_bottom_all.to_csv("sharpe_unsort_bottom_all.csv")
sharpe_unsort_bottom_all

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,HCLTECH Weight,HDFCBANK Weight,HINDALCO Weight,HINDUNILVR Weight,HDFC Weight,ICICIBANK Weight,INFY Weight,JSWSTEEL Weight,KOTAKBANK Weight,NESTLEIND Weight,RELIANCE Weight,SHREECEM Weight,TCS Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,WIPRO Weight
37281,0.274301,0.183314,1.301052,0.045447,0.045473,0.045437,0.045447,0.045453,0.045437,0.045471,0.045445,0.045455,0.045468,0.045458,0.045463,0.045473,0.045447,0.045471,0.045437,0.045455,0.045440,0.045437,0.045453,0.045471,0.045463
37282,0.274300,0.183314,1.301051,0.045448,0.045471,0.045439,0.045448,0.045453,0.045439,0.045468,0.045446,0.045455,0.045466,0.045457,0.045462,0.045471,0.045448,0.045468,0.045439,0.045455,0.045442,0.045439,0.045453,0.045468,0.045462
37283,0.274300,0.183314,1.301049,0.045449,0.045469,0.045441,0.045449,0.045453,0.045441,0.045467,0.045447,0.045455,0.045465,0.045457,0.045461,0.045469,0.045449,0.045467,0.045441,0.045455,0.045443,0.045441,0.045453,0.045467,0.045461
37284,0.274300,0.183314,1.301048,0.045450,0.045467,0.045443,0.045450,0.045453,0.045443,0.045465,0.045448,0.045455,0.045463,0.045457,0.045460,0.045467,0.045450,0.045465,0.045443,0.045455,0.045445,0.045443,0.045453,0.045465,0.045460
37285,0.274300,0.183314,1.301047,0.045450,0.045465,0.045445,0.045450,0.045453,0.045445,0.045464,0.045449,0.045455,0.045462,0.045456,0.045459,0.045465,0.045450,0.045464,0.045445,0.045455,0.045446,0.045445,0.045453,0.045464,0.045459
37286,0.274299,0.183314,1.301046,0.045451,0.045464,0.045446,0.045451,0.045454,0.045446,0.045463,0.045450,0.045455,0.045461,0.045456,0.045459,0.045464,0.045451,0.045463,0.045446,0.045455,0.045447,0.045446,0.045454,0.045463,0.045459
37287,0.274299,0.183314,1.301045,0.045451,0.045463,0.045447,0.045451,0.045454,0.045447,0.045462,0.045450,0.045455,0.045460,0.045456,0.045458,0.045463,0.045451,0.045462,0.045447,0.045455,0.045448,0.045447,0.045454,0.045462,0.045458
37288,0.274299,0.183314,1.301044,0.045452,0.045462,0.045448,0.045452,0.045454,0.045448,0.045461,0.045451,0.045455,0.045460,0.045456,0.045458,0.045462,0.045452,0.045461,0.045448,0.045455,0.045449,0.045448,0.045454,0.045461,0.045458
37289,0.274299,0.183314,1.301043,0.045452,0.045461,0.045449,0.045452,0.045454,0.045449,0.045460,0.045451,0.045455,0.045459,0.045456,0.045457,0.045461,0.045452,0.045460,0.045449,0.045455,0.045450,0.045449,0.045454,0.045460,0.045457
37290,0.274299,0.183314,1.301043,0.045452,0.045460,0.045450,0.045452,0.045454,0.045450,0.045459,0.045452,0.045455,0.045458,0.045455,0.045457,0.045460,0.045452,0.045459,0.045450,0.045455,0.045450,0.045450,0.045454,0.045459,0.045457


In [46]:
sharpe_pc.to_csv('sharpe_ACO_portfolio.csv')

sharpe_pc_sort=sharpe_pc.copy()


# sort

In [47]:
sharpe_pc_sort.sort_values(by=['Sharpe Ratio'],ascending=False,inplace=True)
sharpe_pc_sort.to_csv("sharpe_porfolio_sort.csv")

In [48]:
sharpe_sort_top=sharpe_pc_sort.iloc[1:,0:3].head(11)
sharpe_sort_top.to_csv("sharpe_sort_top.csv")
sharpe_sort_top

,Returns,Standard Deviation,Sharpe Ratio
8011,0.281132,0.177827,1.379612
8012,0.281077,0.177840,1.379206
8013,0.281016,0.177855,1.378742
8014,0.280947,0.177873,1.378215
8015,0.280869,0.177894,1.377616
8016,0.280782,0.177918,1.376937
8017,0.280685,0.177947,1.376168
2492,0.295242,0.188596,1.375654
2493,0.295107,0.188544,1.375319
8018,0.280578,0.177981,1.375300


In [49]:
sharpe_sort_top_all=sharpe_pc_sort.iloc[1:,0:].head(11)
sharpe_sort_top_all.to_csv("sharpe_sort_top_all.csv")
sharpe_sort_top_all

,Returns,Standard Deviation,Sharpe Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,HCLTECH Weight,HDFCBANK Weight,HINDALCO Weight,HINDUNILVR Weight,HDFC Weight,ICICIBANK Weight,INFY Weight,JSWSTEEL Weight,KOTAKBANK Weight,NESTLEIND Weight,RELIANCE Weight,SHREECEM Weight,TCS Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,WIPRO Weight
8011,0.281132,0.177827,1.379612,0.079304,0.055086,0.024815,0.042978,0.085358,0.091412,0.006652,0.018761,0.073249,0.024815,0.024815,0.085358,0.036924,0.006652,0.042978,0.042978,0.006652,0.061141,0.061141,0.024815,0.091412,0.012706
8012,0.281077,0.177840,1.379206,0.079033,0.055009,0.024980,0.042998,0.085039,0.091045,0.006962,0.018974,0.073027,0.024980,0.024980,0.085039,0.036992,0.006962,0.042998,0.042998,0.006962,0.061015,0.061015,0.024980,0.091045,0.012968
8013,0.281016,0.177855,1.378742,0.078728,0.054923,0.025166,0.043020,0.084680,0.090631,0.007311,0.019214,0.072777,0.025166,0.025166,0.084680,0.037068,0.007311,0.043020,0.043020,0.007311,0.060874,0.060874,0.025166,0.090631,0.013263
8014,0.280947,0.177873,1.378215,0.078386,0.054825,0.025375,0.043045,0.084276,0.090166,0.007704,0.019485,0.072495,0.025375,0.025375,0.084276,0.037155,0.007704,0.043045,0.043045,0.007704,0.060715,0.060715,0.025375,0.090166,0.013594
8015,0.280869,0.177894,1.377616,0.078001,0.054716,0.025609,0.043073,0.083822,0.089643,0.008145,0.019788,0.072179,0.025609,0.025609,0.083822,0.037252,0.008145,0.043073,0.043073,0.008145,0.060537,0.060537,0.025609,0.089643,0.013967
8016,0.280782,0.177918,1.376937,0.077570,0.054593,0.025872,0.043105,0.083315,0.089059,0.008639,0.020127,0.071826,0.025872,0.025872,0.083315,0.037360,0.008639,0.043105,0.043105,0.008639,0.060337,0.060337,0.025872,0.089059,0.014383
8017,0.280685,0.177947,1.376168,0.077091,0.054457,0.026164,0.043140,0.082749,0.088407,0.009189,0.020506,0.071432,0.026164,0.026164,0.082749,0.037481,0.009189,0.043140,0.043140,0.009189,0.060115,0.060115,0.026164,0.088407,0.014847
2492,0.295242,0.188596,1.375654,0.078463,0.072877,0.072877,0.061705,0.056119,0.078463,0.056119,0.005845,0.017017,0.005845,0.022603,0.011431,0.072877,0.028189,0.028189,0.078463,0.056119,0.022603,0.039361,0.011431,0.084049,0.039361
2493,0.295107,0.188544,1.375319,0.078251,0.072700,0.072700,0.061600,0.056050,0.078251,0.056050,0.006099,0.017200,0.006099,0.022750,0.011649,0.072700,0.028300,0.028300,0.078251,0.056050,0.022750,0.039400,0.011649,0.083801,0.039400
8018,0.280578,0.177981,1.375300,0.076558,0.054305,0.026489,0.043179,0.082121,0.087684,0.009800,0.020926,0.070994,0.026489,0.026489,0.082121,0.037616,0.009800,0.043179,0.043179,0.009800,0.059868,0.059868,0.026489,0.087684,0.015363


In [50]:
sharpe_sort_bottom=sharpe_pc_sort.iloc[:,0:3].tail(10)
sharpe_sort_bottom.to_csv("sharpe_sort_bottom.csv")
sharpe_sort_bottom

,Returns,Standard Deviation,Sharpe Ratio
31417,0.260294,0.189160,1.186792
4629,0.259010,0.188124,1.186505
34627,0.241955,0.173808,1.186113
4628,0.258932,0.188160,1.185864
34626,0.241589,0.173740,1.184463
34625,0.241262,0.173681,1.182990
34624,0.240973,0.173629,1.181678
34623,0.240717,0.173583,1.180513
34622,0.240491,0.173543,1.179481
34621,0.240292,0.173509,1.178569


In [51]:
sharpe_sort_bottom_all=sharpe_pc_sort.iloc[:,0:3].tail(10)
sharpe_sort_bottom_all.to_csv("sharpe_sort_bottom_all.csv")
sharpe_sort_bottom_all

,Returns,Standard Deviation,Sharpe Ratio
31417,0.260294,0.189160,1.186792
4629,0.259010,0.188124,1.186505
34627,0.241955,0.173808,1.186113
4628,0.258932,0.188160,1.185864
34626,0.241589,0.173740,1.184463
34625,0.241262,0.173681,1.182990
34624,0.240973,0.173629,1.181678
34623,0.240717,0.173583,1.180513
34622,0.240491,0.173543,1.179481
34621,0.240292,0.173509,1.178569


# Sortino

In [ ]:
def antcolony_tuning_sortino(ITERATIONS,Q,EVA_RATE,ANTS):
    sortino_pbest=-1
    #Initializing sortino_pbest(the best fitness value   SHARPE)
    #Initializing the current fitness value
    fitness=0
    #for each iteration
    for iteration in range(ITERATIONS):
        
        #PREPARAING THE PHEROMONE MATRIX WHERE THE COLS=STOCKS AND  ROWS=ANTS
        pheromon=[[0]*stocks for i in range(ANTS+1)]#why (ants+1)?The last ant can update the pheromone values in the last row
        
        # Initializing the pheromone status 
        for i in range(len(pheromon[0])):
            pheromon[0][i]=random.randint(1,15)   #When input stocks varies, this needs to vary accordingly.(Divide number of stocks / 2)
        
        #copying the values and storing it in temp_pher
        temp_pher=pheromon[0]
        
        #Making sure that the total amount of pheromone equals 1 
        weights=np.array(BALANCE(temp_pher))
        
        #calculating annulaised portfolio return
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days 
        
        #calculating portfolio varience wrt calculating sharpe ratio
        varso=np.dot(weights.T,np.dot(covmatso,weights))   
        
        #portfolio risk
        volatility_temp = np.sqrt(varso)      
        
        #Calculating fitness value(ie sortino ratio)
        fitness = ratio(returns_temp,volatility_temp,risk_free_rate)
        
        #Initializing the intial fitness value as the best fitness value(sortino_pbest)
        if sortino_pbest==-1:
            sortino_pbest=fitness
        
        #list
        path=[]
      
        #for each ant
        for ant in range(ANTS-1):
            
            #find the total pheromone 
            total=sum(pheromon[ant])
            
            #Initializing probability
            probability=pheromon[ant][:] 
            
            #finding probability of each stocks pheromone 
            for p in range(len(probability)):
                probability[p]=(probability[p]/total)
                
            #Trying to select stocks in decreasing order based on their pheromone level and storing the stock order in a list(path)
            for stock in range(stocks):
                select=probability.index(max(probability))
                probability[select]=-math.inf
                path.append(select)
            
            #Updating the pheromone level of each stock for the next ant 
            #Formula: old pheromone level * (1-eva_rate) + Q * (fitness/pbest) where Q is fixed amount of pheromone
            for s in path:
                pheromon[ant+1][s]=pheromon[ant][s]*(1-EVA_RATE)+Q*(fitness/sortino_pbest)
            
            
            #making sure that the updated pheromon adds upto 1
            temp_pher=pheromon[ant+1]
            weights=np.array(BALANCE(temp_pher))
            returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
            varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sharpe ratio
            volatility_temp = np.sqrt(varso)                              #portfolio risk
            fitness = ratio(returns_temp,volatility_temp,risk_free_rate)  #calculating sharpe ratio
            
            #comparing the old fitness value with the updated fitness value
            # explore on scape condition paper ref
            if(fitness>sortino_pbest):
                
                #if the updated fitness value is better than the previous, change sortino_pbest to present fitness value
                sortino_pbest=fitness
                
                #remembering the weights of the best portfolio
                global_warr_sortino=weights.tolist()
        #sortino_portfolio_return.append(returns_temp)
        #sortino_portfolio_risk.append(volatility_temp)
        #sortino_portfolio_soratio.append(fitness)
        #sortino_portfolio_stockWeights.append(weights)
    return sortino_pbest


In [ ]:
def objective(trial):
    ITERATIONS=trial.suggest_int('ITERATIONS',2,500)#range is temp,earlier (1,1000)
    Q=trial.suggest_float('Q',0.0,1.0)
    EVA_RATE=trial.suggest_float('EVA_RATE',0.00,1.00)
    ANTS=trial.suggest_int('ANTS',2,500)#range is temp,earlier (1,1000)
    return antcolony_tuning_sortino(int(ITERATIONS),Q,EVA_RATE,int(ANTS))


In [ ]:
sortino_study=optuna.create_study(direction='maximize')
sortino_study.optimize(objective,n_trials=5)

[I 2021-11-09 14:33:43,725] A new study created in memory with name: no-name-9279b094-17e7-4f59-98ea-0727502e38cb
[I 2021-11-09 14:33:47,431] Trial 0 finished with value: -13.02787413128197 and parameters: {'ITERATIONS': 181, 'Q': 0.2126166736228623, 'EVA_RATE': 0.7768917558107318, 'ANTS': 19}. Best is trial 0 with value: -13.02787413128197.
[I 2021-11-09 14:34:09,329] Trial 1 finished with value: -13.034678512526453 and parameters: {'ITERATIONS': 181, 'Q': 0.8557771232298401, 'EVA_RATE': 0.7214370128533921, 'ANTS': 78}. Best is trial 0 with value: -13.02787413128197.
[I 2021-11-09 14:34:18,081] Trial 2 finished with value: -13.058982818584928 and parameters: {'ITERATIONS': 185, 'Q': 0.9161007814220569, 'EVA_RATE': 0.5228342304699373, 'ANTS': 39}. Best is trial 0 with value: -13.02787413128197.
[I 2021-11-09 14:34:45,586] Trial 3 finished with value: -12.964734167484135 and parameters: {'ITERATIONS': 190, 'Q': 0.4460995487764283, 'EVA_RATE': 0.17068827931945185, 'ANTS': 88}. Best is tr

In [ ]:
hptuning=sortino_study.trials_dataframe()
hptuning.to_csv("sortino_trial0.csv")
best=sortino_study.best_params
best

{'ANTS': 4,
 'EVA_RATE': 0.21381692108318961,
 'ITERATIONS': 182,
 'Q': 0.6845045271134248}

In [56]:
ITERATIONS=419#int(best['ITERATIONS'])
Q=0.0400026#best['Q']
EVA_RATE=0.128567#best['EVA_RATE']
ANTS=90#int(best['ANTS'])

In [59]:
global_warr_sortino=[]
sortino_portfolio_return=[]
sortino_portfolio_risk=[]
sortino_portfolio_soratio=[]
sortino_portfolio_stockWeights=[]

In [57]:
def antcolony_sortino(ITERATIONS,Q,EVA_RATE,ANTS):
    sortino_pbest=-1
    #Initializing sortino_pbest(the best fitness value   SHARPE)
    #Initializing the current fitness value
    fitness=0
    #for each iteration
    for iteration in range(ITERATIONS):
        
        #PREPARAING THE PHEROMONE MATRIX WHERE THE COLS=STOCKS AND  ROWS=ANTS
        pheromon=[[0]*stocks for i in range(ANTS+1)]#why (ants+1)?The last ant can update the pheromone values in the last row
        
        # Initializing the pheromone status 
        for i in range(len(pheromon[0])):
            pheromon[0][i]=random.randint(1,15)   #When input stocks varies, this needs to vary accordingly.(Divide number of stocks / 2)
        
        #copying the values and storing it in temp_pher
        temp_pher=pheromon[0]
        
        #Making sure that the total amount of pheromone equals 1 
        weights=np.array(BALANCE(temp_pher))
        
        #calculating annulaised portfolio return
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days 
        
        #calculating portfolio varience wrt calculating sharpe ratio
        varso=np.dot(weights.T,np.dot(covmatso,weights))   
        
        #portfolio risk
        volatility_temp = np.sqrt(varso)      
        
        #Calculating fitness value(ie sortino ratio)
        fitness = ratio(returns_temp,volatility_temp,risk_free_rate)
        
        #Initializing the intial fitness value as the best fitness value(sortino_pbest)
        if sortino_pbest==-1:
            sortino_pbest=fitness
        
        #list
        path=[]
      
        #for each ant
        for ant in range(ANTS-1):
            
            #find the total pheromone 
            total=sum(pheromon[ant])
            
            #Initializing probability
            probability=pheromon[ant][:] 
            
            #finding probability of each stocks pheromone 
            for p in range(len(probability)):
                probability[p]=(probability[p]/total)
                
            #Trying to select stocks in decreasing order based on their pheromone level and storing the stock order in a list(path)
            for stock in range(stocks):
                select=probability.index(max(probability))
                probability[select]=-math.inf
                path.append(select)
            
            #Updating the pheromone level of each stock for the next ant 
            #Formula: old pheromone level * (1-eva_rate) + Q * (fitness/pbest) where Q is fixed amount of pheromone
            for s in path:
                pheromon[ant+1][s]=pheromon[ant][s]*(1-EVA_RATE)+Q*(fitness/sortino_pbest)
            
            
            #making sure that the updated pheromon adds upto 1
            temp_pher=pheromon[ant+1]
            weights=np.array(BALANCE(temp_pher))
            returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
            varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sharpe ratio
            volatility_temp = np.sqrt(varso)                              #portfolio risk
            fitness = ratio(returns_temp,volatility_temp,risk_free_rate)  #calculating sharpe ratio
            
            #comparing the old fitness value with the updated fitness value
            # explore on scape condition paper ref
            if(fitness>sortino_pbest):
                
                #if the updated fitness value is better than the previous, change sortino_pbest to present fitness value
                sortino_pbest=fitness
                
                #remembering the weights of the best portfolio
                global_warr_sortino=weights.tolist()
            sortino_portfolio_return.append(returns_temp)
            sortino_portfolio_risk.append(volatility_temp)
            sortino_portfolio_soratio.append(fitness)
            sortino_portfolio_stockWeights.append(weights)
    return sortino_pbest


In [60]:
sortino_tuned=antcolony_sortino(ITERATIONS,Q,EVA_RATE,ANTS)

In [61]:
sortino_tuned

2.2148015961140124

In [77]:
sortino_portfolio = {'Returns' : sortino_portfolio_return, 'Standard Deviation' : sortino_portfolio_risk,  'Sortino Ratio' : sortino_portfolio_soratio}  

for counter,symbol in enumerate(df.columns):
  sortino_portfolio[symbol + " Weight"] = [Weight[counter] for Weight in sortino_portfolio_stockWeights]
sortino_pc = pd.DataFrame(sortino_portfolio)
sortino_optimal=sortino_pc.iloc[sortino_pc['Sortino Ratio'].idxmax()]
sortino_optimal=pd.DataFrame(sortino_optimal)
sortino_optimal.to_csv("sortino_optimal.csv")
sortino_optimal
sortino_pc.shape

(37291, 25)

## not sort 

In [64]:
sortino_unsort_top=sortino_pc.iloc[:,0:3].head(10)
sortino_unsort_top.to_csv("sortino_unsort_top.csv")

In [65]:
sortino_unsort_top_all=sortino_pc.head(10)
sortino_unsort_top_all.to_csv("sortino_unsort_top_all.csv")

In [66]:
sortino_unsort_bottom=sortino_pc.iloc[:,0:3].tail(10)
sortino_unsort_bottom.to_csv("sortino_unsort_bottom.csv")

In [67]:
sortino_unsort_bottom_all=sortino_pc.tail(10)
sortino_unsort_bottom_all.to_csv("sortino_unsort_bottom_all.csv")

In [68]:
sortino_pc.to_csv('sortino_ACO_portfolio.csv')

sortino_pc_sort=sortino_pc.copy()

## sort sortino

In [69]:
sortino_pc_sort.sort_values(by=['Sortino Ratio'],ascending=False,inplace=True)
sortino_pc_sort.to_csv("sortino_porfolio_sort.csv")

In [70]:
sortino_sort_top=sortino_pc_sort.iloc[1:,0:3].head(11)
sortino_sort_top.to_csv("sortino_sort_top.csv")
sortino_sort_top

,Returns,Standard Deviation,Sortino Ratio
18691,0.289450,0.114562,2.214096
18692,0.289360,0.114562,2.213294
18693,0.289258,0.114563,2.212385
18694,0.289143,0.114564,2.211356
18695,0.289013,0.114566,2.210193
18696,0.288866,0.114568,2.208882
18697,0.288702,0.114570,2.207408
18698,0.288517,0.114572,2.205754
18699,0.288312,0.114575,2.203905
18700,0.288084,0.114578,2.201844


In [71]:
sortino_sort_top_all=sortino_pc_sort.iloc[1:,0:].head(11)
sortino_sort_top_all.to_csv("sortino_sort_top_all.csv")
sortino_sort_top_all

,Returns,Standard Deviation,Sortino Ratio,ASIANPAINT Weight,BAJFINANCE Weight,BAJAJFINSV Weight,BRITANNIA Weight,DIVISLAB Weight,HCLTECH Weight,HDFCBANK Weight,HINDALCO Weight,HINDUNILVR Weight,HDFC Weight,ICICIBANK Weight,INFY Weight,JSWSTEEL Weight,KOTAKBANK Weight,NESTLEIND Weight,RELIANCE Weight,SHREECEM Weight,TCS Weight,TATASTEEL Weight,TECHM Weight,TITAN Weight,WIPRO Weight
18691,0.289450,0.114562,2.214096,0.036453,0.063458,0.049955,0.054456,0.067959,0.049955,0.013948,0.058957,0.067959,0.009448,0.022950,0.058957,0.040954,0.045455,0.058957,0.063458,0.022950,0.063458,0.049955,0.022950,0.058957,0.018449
18692,0.289360,0.114562,2.213294,0.036506,0.063351,0.049929,0.054403,0.067825,0.049929,0.014136,0.058877,0.067825,0.009662,0.023084,0.058877,0.040980,0.045455,0.058877,0.063351,0.023084,0.063351,0.049929,0.023084,0.058877,0.018610
18693,0.289258,0.114563,2.212385,0.036567,0.063230,0.049898,0.054342,0.067673,0.049898,0.014348,0.058786,0.067673,0.009904,0.023236,0.058786,0.041011,0.045455,0.058786,0.063230,0.023236,0.063230,0.049898,0.023236,0.058786,0.018792
18694,0.289143,0.114564,2.211356,0.036636,0.063093,0.049864,0.054274,0.067502,0.049864,0.014588,0.058683,0.067502,0.010178,0.023407,0.058683,0.041045,0.045455,0.058683,0.063093,0.023407,0.063093,0.049864,0.023407,0.058683,0.018998
18695,0.289013,0.114566,2.210193,0.036713,0.062938,0.049825,0.054196,0.067309,0.049825,0.014859,0.058567,0.067309,0.010488,0.023600,0.058567,0.041084,0.045455,0.058567,0.062938,0.023600,0.062938,0.049825,0.023600,0.058567,0.019230
18696,0.288866,0.114568,2.208882,0.036800,0.062764,0.049782,0.054109,0.067091,0.049782,0.015163,0.058436,0.067091,0.010836,0.023818,0.058436,0.041127,0.045455,0.058436,0.062764,0.023818,0.062764,0.049782,0.023818,0.058436,0.019491
18697,0.288702,0.114570,2.207408,0.036898,0.062568,0.049733,0.054011,0.066847,0.049733,0.015505,0.058290,0.066847,0.011227,0.024062,0.058290,0.041176,0.045455,0.058290,0.062568,0.024062,0.062568,0.049733,0.024062,0.058290,0.019784
18698,0.288517,0.114572,2.205754,0.037007,0.062350,0.049678,0.053902,0.066573,0.049678,0.015888,0.058126,0.066573,0.011665,0.024336,0.058126,0.041231,0.045455,0.058126,0.062350,0.024336,0.062350,0.049678,0.024336,0.058126,0.020112
18699,0.288312,0.114575,2.203905,0.037129,0.062105,0.049617,0.053780,0.066268,0.049617,0.016315,0.057943,0.066268,0.012153,0.024641,0.057943,0.041292,0.045455,0.057943,0.062105,0.024641,0.062105,0.049617,0.024641,0.057943,0.020478
18700,0.288084,0.114578,2.201844,0.037265,0.061834,0.049549,0.053644,0.065929,0.049549,0.016790,0.057739,0.065929,0.012695,0.024980,0.057739,0.041360,0.045455,0.057739,0.061834,0.024980,0.061834,0.049549,0.024980,0.057739,0.020885


In [72]:
sortino_sort_bottom=sortino_pc_sort.iloc[:,0:3].tail(10)
sortino_sort_bottom.to_csv("sortino_sort_bottom.csv")
sortino_sort_bottom

,Returns,Standard Deviation,Sortino Ratio
24388,0.259096,0.119346,1.871006
12555,0.241916,0.110223,1.869994
24387,0.259007,0.119376,1.869779
24386,0.258927,0.119403,1.868699
12554,0.241551,0.110198,1.867112
12553,0.241227,0.110176,1.864538
12552,0.240940,0.110157,1.862248
12551,0.240686,0.110141,1.860214
12550,0.240462,0.110127,1.858413
12549,0.240264,0.110115,1.856820


In [73]:
sortino_sort_bottom_all=sortino_pc_sort.iloc[:,0:3].tail(10)
sortino_sort_bottom_all.to_csv("sortino_sort_bottom_all.csv")
sortino_sort_bottom_all

,Returns,Standard Deviation,Sortino Ratio
24388,0.259096,0.119346,1.871006
12555,0.241916,0.110223,1.869994
24387,0.259007,0.119376,1.869779
24386,0.258927,0.119403,1.868699
12554,0.241551,0.110198,1.867112
12553,0.241227,0.110176,1.864538
12552,0.240940,0.110157,1.862248
12551,0.240686,0.110141,1.860214
12550,0.240462,0.110127,1.858413
12549,0.240264,0.110115,1.856820


In [74]:

#tables in word doc also(final table)
